In [ ]:
from google.colab import files
import collections
import numpy as np
import pandas as pd
from sklearn import preprocessing
import torch.nn.functional as F
import torch

In [ ]:
files = files.upload()

Saving ptb.train.txt to ptb.train.txt


In [ ]:
def get_lines():
  with open('ptb.train.txt', 'r') as f:
    lines = f.readlines()
  return lines

lines = get_lines()
print(lines[0:1000])
print(len(lines))

[' aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter \n', ' pierre <unk> N years old will join the board as a nonexecutive director nov. N \n', ' mr. <unk> is chairman of <unk> n.v. the dutch publishing group \n', ' rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate \n', ' a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported \n', ' the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said \n', ' <unk> inc. the unit of new york-based <unk> corp. that makes kent cigarettes stopped using <unk> in its <unk> cigarette filters in N \n'

In [ ]:
def tokenize(lines, token):
  if token == 'word':
    Tokens = [line.split() for line in lines]
  elif token == 'char':
    Tokens = [list(line) for line in lines]

  return Tokens

Tokens = tokenize(lines, token='char')
print(Tokens[0:100])


[[' ', 'a', 'e', 'r', ' ', 'b', 'a', 'n', 'k', 'n', 'o', 't', 'e', ' ', 'b', 'e', 'r', 'l', 'i', 't', 'z', ' ', 'c', 'a', 'l', 'l', 'o', 'w', 'a', 'y', ' ', 'c', 'e', 'n', 't', 'r', 'u', 's', 't', ' ', 'c', 'l', 'u', 'e', 't', 't', ' ', 'f', 'r', 'o', 'm', 's', 't', 'e', 'i', 'n', ' ', 'g', 'i', 't', 'a', 'n', 'o', ' ', 'g', 'u', 't', 'e', 'r', 'm', 'a', 'n', ' ', 'h', 'y', 'd', 'r', 'o', '-', 'q', 'u', 'e', 'b', 'e', 'c', ' ', 'i', 'p', 'o', ' ', 'k', 'i', 'a', ' ', 'm', 'e', 'm', 'o', 't', 'e', 'c', ' ', 'm', 'l', 'x', ' ', 'n', 'a', 'h', 'b', ' ', 'p', 'u', 'n', 't', 's', ' ', 'r', 'a', 'k', 'e', ' ', 'r', 'e', 'g', 'a', 't', 't', 'a', ' ', 'r', 'u', 'b', 'e', 'n', 's', ' ', 's', 'i', 'm', ' ', 's', 'n', 'a', 'c', 'k', '-', 'f', 'o', 'o', 'd', ' ', 's', 's', 'a', 'n', 'g', 'y', 'o', 'n', 'g', ' ', 's', 'w', 'a', 'p', 'o', ' ', 'w', 'a', 'c', 'h', 't', 'e', 'r', ' ', '\n'], [' ', 'p', 'i', 'e', 'r', 'r', 'e', ' ', '<', 'u', 'n', 'k', '>', ' ', 'N', ' ', 'y', 'e', 'a', 'r', 's', ' ', 

In [ ]:
def flatten(Tokens):
  return [item for i in Tokens for item in i]

tokens = flatten(Tokens)
print(tokens[0:100])
print(len(tokens))

[' ', 'a', 'e', 'r', ' ', 'b', 'a', 'n', 'k', 'n', 'o', 't', 'e', ' ', 'b', 'e', 'r', 'l', 'i', 't', 'z', ' ', 'c', 'a', 'l', 'l', 'o', 'w', 'a', 'y', ' ', 'c', 'e', 'n', 't', 'r', 'u', 's', 't', ' ', 'c', 'l', 'u', 'e', 't', 't', ' ', 'f', 'r', 'o', 'm', 's', 't', 'e', 'i', 'n', ' ', 'g', 'i', 't', 'a', 'n', 'o', ' ', 'g', 'u', 't', 'e', 'r', 'm', 'a', 'n', ' ', 'h', 'y', 'd', 'r', 'o', '-', 'q', 'u', 'e', 'b', 'e', 'c', ' ', 'i', 'p', 'o', ' ', 'k', 'i', 'a', ' ', 'm', 'e', 'm', 'o', 't', 'e']
5101618


In [ ]:
def unique_tokens(tokens):
  unique_tok = []
  for i in tokens:
    if i not in unique_tok:
      unique_tok.append(i)

  return unique_tok

uniq_tokens = unique_tokens(tokens)
print(uniq_tokens[0:100])
print(len(uniq_tokens))

[' ', 'a', 'e', 'r', 'b', 'n', 'k', 'o', 't', 'l', 'i', 'z', 'c', 'w', 'y', 'u', 's', 'f', 'm', 'g', 'h', 'd', '-', 'q', 'p', 'x', '\n', '<', '>', 'N', 'j', 'v', '.', "'", '1', '9', '5', '0', '&', '$', '3', '2', '4', '8', '6', '7', '#', '\\', '/', '*']
50


In [ ]:
character_dict = {}
for e, char in enumerate(uniq_tokens):
  character_dict[char] = e
print(character_dict)

{' ': 0, 'a': 1, 'e': 2, 'r': 3, 'b': 4, 'n': 5, 'k': 6, 'o': 7, 't': 8, 'l': 9, 'i': 10, 'z': 11, 'c': 12, 'w': 13, 'y': 14, 'u': 15, 's': 16, 'f': 17, 'm': 18, 'g': 19, 'h': 20, 'd': 21, '-': 22, 'q': 23, 'p': 24, 'x': 25, '\n': 26, '<': 27, '>': 28, 'N': 29, 'j': 30, 'v': 31, '.': 32, "'": 33, '1': 34, '9': 35, '5': 36, '0': 37, '&': 38, '$': 39, '3': 40, '2': 41, '4': 42, '8': 43, '6': 44, '7': 45, '#': 46, '\\': 47, '/': 48, '*': 49}


In [ ]:
ptb_numerical = [character_dict[char] for char in tokens]
print(len(ptb_numerical))

5101618


In [ ]:
print("".join([uniq_tokens[idx] for idx in ptb_numerical[:100]]))

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote


In [ ]:
def one_hot_data(numerical_list, vocab_size=50):
    result = torch.zeros((len(numerical_list), vocab_size))
    for i, idx in enumerate(numerical_list):
        result[i, idx] = 1.0
    return result

In [ ]:
print(one_hot_data(ptb_numerical[:2]))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [ ]:
def textify(embedding):
    result = ""
    indices = torch.argmax(embedding, axis=1)
    for idx in indices:
        result += uniq_tokens[int(idx)]
    return result

In [ ]:
seq_length = 64
num_samples = (len(ptb_numerical) - 1) // seq_length
dataset = one_hot_data(ptb_numerical[:num_samples * seq_length]).reshape(num_samples, seq_length, len(uniq_tokens))
dataset.shape

torch.Size([79712, 64, 50])

In [ ]:
batch_size = 32
num_batches = len(dataset) // batch_size
train_iter = dataset[:num_batches * batch_size].reshape((batch_size, num_batches, seq_length, len(uniq_tokens)))
train_iter = train_iter.swapaxes(0, 1)
train_iter = train_iter.swapaxes(1, 2)
train_iter.shape

torch.Size([2491, 64, 32, 50])

In [ ]:
labels = one_hot_data(ptb_numerical[1:num_samples * seq_length + 1]).reshape(batch_size, num_batches, seq_length, len(uniq_tokens))
labels = labels.swapaxes(0, 1)
labels = labels.swapaxes(1, 2)
labels.shape

torch.Size([2491, 64, 32, 50])

In [ ]:
print(textify(train_iter[10, :, 3]))
print(textify(labels[10, :, 3]))

ell based in los angeles makes and distributes electronic comput
ll based in los angeles makes and distributes electronic compute


In [ ]:
class RNN(torch.nn.Module):
  def __init__(self):
    
    self.W_xh = torch.normal(0, 0.01, (50, 256), requires_grad=True)
    self.W_hh = torch.normal(0, 0.01, (256, 256), requires_grad=True)
    self.b_h = torch.zeros(1, 256, requires_grad=True)

    self.W_hq = torch.normal(0, 0.01, (256, 50), requires_grad=True)
    self.b_q = torch.zeros(1, 50, requires_grad=True)
    self.params = [self.W_xh, self.W_hh, self.b_h, self.W_hq, self.b_q]
    for param in self.params:
      param.requires_grad_(True)
  
  def net(self, input, state):
    hidden_act = state
    outputs = []
    for X in input:
      hidden = torch.matmul(X, self.W_xh) + torch.matmul(hidden_act, self.W_hh) + self.b_h
      hidden_act = torch.nn.Tanh(hidden)
      outputs.append(torch.matmul(hidden_act, self.W_hq) + self.b_q)

      return output, hidden_act

  def init_rnn_hidden(self):
    return torch.zeros((1, 256))

In [ ]:

W_xh = torch.normal(0, 0.01, (50, 256), requires_grad=True)
W_hh = torch.normal(0, 0.01, (256, 256), requires_grad=True)
b_h = torch.zeros(1, 256, requires_grad=True)

W_hq = torch.normal(0, 0.01, (256, 50), requires_grad=True)
b_q = torch.zeros(1, 50, requires_grad=True)
params = [W_xh, W_hh, b_h, W_hq, b_q]

In [ ]:
def net(input, state):
  W_xh, W_hh, b_h, W_hq, b_q = params
  hidden_act = state
  outputs = []
  for X in input:
    hidden = torch.matmul(X, W_xh) + torch.matmul(hidden_act, W_hh) + b_h
    hidden_act = torch.tanh(hidden)
    outputs.append(softmax(torch.matmul(hidden_act, W_hq) + b_q))

  return (outputs, hidden_act)

In [ ]:
def init_rnn_hidden():
  return torch.zeros((1, 256))

In [ ]:
def cross_entropy(y_hat, y):
  return -torch.mean(torch.sum(y * torch.log(y_hat)))

In [ ]:
def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = 0.
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
    return total_loss / len(outputs)

In [ ]:
def softmax(X):
  lin = (X - torch.max(X).reshape((-1, 1)))
  X_exp = torch.exp(lin)
  partition = X_exp.sum(1, keepdim=True)
  return X_exp / partition

In [ ]:
def grad_clipping(net, theta):
    """Clip the gradient."""
    params = net
    norm = torch.sqrt(sum(torch.sum((p.grad**2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [ ]:
rnn = net
criterion = average_ce_loss
params = params
lr = 0.005
optimizer = torch.optim.SGD(params, lr)
num_epochs = 75


for epoch in range(num_epochs):
  state = init_rnn_hidden()
  for i in range(num_batches):
    input = train_iter[i]
    train_labels = labels[i]
    state = state.detach()
    optimizer.zero_grad()
    y_hat, state = rnn(input, state)
    l = criterion(y_hat, train_labels)
    l.sum().backward()
    grad_clipping(params, 1)

    optimizer.step()

  with torch.no_grad():
    l_loss = criterion(y_hat, train_labels)
    print(f'loss on epoch {epoch} was {l_loss}')
    print(predict('on the other hand', 512))

loss on epoch 0 was 46.17603302001953
on the other hands actek the jondy <unk> deqlion n't offecinge coutory on a sais monily <unk> welly machistriganca some chinfecred equirminicy foldous teessoly ackmonged a rercounts of the isc. are <unk> rvile it as ffic exal furm 
 the operation 
 yet cissa loo n't natroum drenclisis by hred proprem traders <unk> plowhear than the crad is i cust covment 
 befines manky 's ssick estand branks stust kidst on the rigk that what n't quarten a <unk> becorn in the s0uple about N and <unk> last week in plyent <unk> estivitied lef
loss on epoch 1 was 46.140037536621094
on the other hand hould pelt an oney out in the medean mondar the company s op their dutagr ma. ' feal belefined aurbored to <unk> <unk> <unk> a bradit will might covilinuly presidest comp. an west ling the dewier-dackveblezres wonth labrets and thoslakd <unk> <unk> ffom ewothe deely 
 the twad recondented market that siges a president blays sco vailies bundgerilt was deacing <unk> goinds a

In [ ]:
def predict(prefix, num_chars):
  string = prefix
  sample_state = init_rnn_hidden()
  string_numerical = [character_dict[char] for char in prefix]
  input = one_hot_data(string_numerical)
  
  for i in range(num_chars):
    outputs, sample_state = rnn(input, sample_state)
    choice = np.random.choice(50, p=fix_p(np.asarray(outputs[-1][0])))
    string += uniq_tokens[choice]
    input = one_hot_data([choice])
  return string

In [ ]:
def fix_p(p):
    if p.sum() != 1.0:
        p = p*(1./p.sum())
    return p

In [ ]:
my_generator = np.random.default_rng()

In [ ]:
prefix = 'my name is'
string = prefix
sample_state = init_rnn_hidden()
string_numerical = [character_dict[char] for char in prefix]
input = one_hot_data(string_numerical)

for i in range(20):
  with torch.no_grad():
    outputs, sample_state = rnn(input, sample_state)
    choice = np.random.choice(50, p=fix_p(np.asarray(outputs[-1][0])))
    string += uniq_tokens[choice]
    input = one_hot_data([choice])

In [ ]:
a = fix_p(outputs[-1][0].sum())
a

tensor(1.)